In [2]:
!pip install pyautogui 

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.53-py3-none-any.whl size=36589 sha256=b74f23ebdd8ca1f86ca7eca1b06bf019ba89ba9bb4504dbe6184862a07ced674
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\54\04\f7\55704a9d374ed923785a4fdc0ef00151fa25306b3b93345532
  Created wheel for pymsgbox: filename=PyMsgBox-1.0.9-py3-none-any.whl size=7420 sha256=87

In [19]:
import pdb

import pyautogui

# selenium
import selenium
import pandas as pd
from selenium import webdriver


# BeautifulSoup
from bs4 import BeautifulSoup
import requests

# add time
import time

#to send keys from keyboard
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import NoSuchElementException

In [20]:
def open_url(url):
    """function to open the entered url in browser"""
    global driver
    # first, connect to the webdriver
    driver=webdriver.Chrome(r'C:\Users\admin\Downloads\chromedriver_win32\chromedriver.exe')
    # maximize window
    driver.maximize_window()
    #enter the url
    driver.get(url)
    time.sleep(3)
    # handle 'Privacy Error' from Chrome
    try:
        driver.find_element_by_xpath("//div[@class='interstitial-wrapper']//following::button[@id='details-button']").click()
        
        time.sleep(2)
        
        driver.find_element_by_xpath("//*[@id='proceed-link']").click()
        
        time.sleep(2)
        
    except NoSuchElementException:
        pass

### Q.1. amazon.in , any product entered by user should get searched.

In [22]:
product_name=input("Which amazon product do you have in mind ?:")

url='http://www.amazon.in.'

open_url(url)

# give some time to load the webpage
time.sleep(2)

# get web element for amazon search bar
search_wbe=driver.find_element_by_xpath("//input[@class='nav-input nav-progressive-attribute']")
#enter the product into search bar
search_wbe.send_keys(product_name,Keys.ENTER)

# give some time to loda the search results
time.sleep(2)

Which amazon product do you have in mind ?:guitar


### Q.2 - In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product vertical.
### Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [23]:
def amzn_search(input_prompt):
    """this function will search a particular product category on amazon.in website"""
    
    open_url('http://www.amazon.in.')

    # give some time to load the webpage
    time.sleep(3)

    # get web element for amazon search bar
    search_wbe=driver.find_element_by_xpath("//input[@class='nav-input nav-progressive-attribute']")
    #enter the product into search bar
    search_wbe.send_keys(str(input_prompt),Keys.ENTER) 

    # give some time to loda the search results
    time.sleep(3)

In [24]:
def fetch_product():
    # first get all product tiles from current webpage
    #get product tiles webelements
    global tiles_wbe
    tiles_wbe=[]
    tiles_wbe.clear()
    
    tiles_wbe=driver.find_elements_by_xpath("//div[contains(@data-asin,'B0') and @data-component-type='s-search-result']")
    
    # click on each tile one by one and get product info
    for i in range(0,len(tiles_wbe)):
        
        temp_wbe=tiles_wbe[i]
        #add time
        time.sleep(2)
        temp_wbe.click()
        try:
            # switch to new window
            driver.switch_to.window(driver.window_handles[1])
        except IndexError:
            temp_wbe=tiles_wbe[i]
            time.sleep(2)
            temp_wbe.click()
            driver.switch_to.window(driver.window_handles[1])
        # add timer
        time.sleep(3)
        
        # below part is to enter pincode 
        #=========================================================
        # get "enter pincode" button webelement
        pincode_wbe=driver.find_element_by_xpath("//div[@id='contextualIngressPtLabel_deliveryShortLine']")
        
        if pincode_wbe.text=='Select delivery location':
            #click on 'enter pincode' button
            pincode_wbe.click()
            #add time to load pincode sub-window
            time.sleep(2)
            #get the webelement for pincode entry bar
            pin_bar_wbe=driver.find_element_by_xpath("//input[@class='GLUX_Full_Width a-declarative']")
            #enter into pincode bar
            pin_bar_wbe.send_keys('110091')
            #get webelement for pincode 'Apply' button
            apply_btn=driver.find_element_by_xpath("//input[@aria-labelledby='GLUXZipUpdate-announce']")
            #click on pincode 'Apply' button
            apply_btn.click()
            #add time to load the webpage after pincode enter
            time.sleep(3)
          #=====================================================  
        
        time.sleep(3)
        # calling the function to fetch data from current webpage
        fetch_data()
        
        # give some time to fetch data
        time.sleep(3)
        
        # close the particular product tab
        driver.close()
        
        # switch driver to main window with search results
        driver.switch_to.window(driver.window_handles[0])
        
        time.sleep(2)

In [25]:
def fetch_data():
    """this function gets required data from the current product webpage"""
    
    # get brand name
    brand_wbe=driver.find_element_by_xpath("//span[@id='productTitle']")
    brands.append(brand_wbe.text.split(' ')[0])
    # get name of the product
    products.append(brand_wbe.text)
    # get the rating
    try:
        rating_wbe=driver.find_element_by_xpath("//span[@data-hook='rating-out-of-text']") # only first instance of webelement from this page is required
        ratings.append(rating_wbe.text)
    except NoSuchElementException:
        ratings.append('--')

    # get number of ratings
    try:
        number_ratings_wbe=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']//following::span")
        number_ratings.append(number_ratings_wbe.text.split(" ")[0])
    except NoSuchElementException:
        number_ratings.append('--')
 
    # get price 
    try:
        price_wbe=driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
        prices.append(price_wbe.text)
    except NoSuchElementException:
        try:
            price_wbe=driver.find_element_by_xpath("//span[@id='priceblock_dealprice']")
            prices.append(price_wbe.text+"(Deal price)")
        except NoSuchElementException:
            try:
                price_wbe=driver.find_element_by_xpath("//span[@class='a-price-whole']")
                prices.append(price_wbe.text+"(Prime member price)")
            except NoSuchElementException:
                prices.append('--')
    

    # get return/exchange data
    try:
        xchange_wbe=driver.find_element_by_xpath("//a[@class='a-size-small a-link-normal a-text-normal']")
        xchanges.append(xchange_wbe.text)
    except NoSuchElementException:
        xchanges.append('--')
         

    # get expected delivery webelement
    try:
        xp_del_wbe=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']")
        if xp_del_wbe.text!='':
            xp_deliveries.append(xp_del_wbe.text)
        else:
            xp_deliveries.append("Currently not available")
    except NoSuchElementException:
        xp_deliveries.append('--')
         

    # get product availability
    try:
        prod_avail_wbe=driver.find_element_by_xpath("//div[@id='availability']")
        prod_avail.append(prod_avail_wbe.text)
    except NoSuchElementException:
        prod_avail.append('--')     

    # get other details
    try:
        other_details_wbe=driver.find_element_by_xpath("//div[@id='featurebullets_feature_div']")
        other_details.append(other_details_wbe.text.replace("\n","."))
    except NoSuchElementException:
        other_details.append('--')
   
    # get product url
    prod_url.append(driver.current_url)

In [26]:
brands=[]
brands.clear()

products=[]
products.clear()

ratings=[]
ratings.clear()

number_ratings=[]
number_ratings.clear()

prices=[]
prices.clear()

xchanges=[]
xchanges.clear()

xp_deliveries=[]
xp_deliveries.clear()

prod_avail=[]
prod_avail.clear()

other_details=[]
other_details.clear()

prod_url=[]
prod_url.clear()

page_urls=[]
page_urls.clear()
input_prompt=input("Which amazon product do you have in mind ?:")

amzn_search(input_prompt)

#add time
time.sleep(3)

# get web element for webpage number button
page_wbe=driver.find_elements_by_xpath("//ul[@class='a-pagination']//following::a")

# get urls of 1st 3 pages
for i in page_wbe:
    temp_url= i.get_attribute('href')
    page_urls.append(temp_url)
    if len(page_urls)==3:
        break

counter=1
# get product info by iterating over each page
for url in page_urls:
    if counter == 1: # first page check
        time.sleep(2)
        fetch_product() # get webelemets for product titles, then click on each title and open the product in new tab and get info.
        counter=2
    elif counter != 1: # first page scraping complete
        time.sleep(2)
        # open the next webpage in current tab
        driver.get(url)
        
        # give time to load
        time.sleep(3)
        
        # fetch data from current webpage
        fetch_product()
        
        # time
        time.sleep(3)

Which amazon product do you have in mind ?:guitar


In [27]:
len(brands)

0

In [28]:
amzn_3pg=pd.DataFrame({})
amzn_3pg['Brand']=brands
amzn_3pg['Product']=products
amzn_3pg['Rating']=ratings
amzn_3pg['Number of ratings']=number_ratings
amzn_3pg['Price']=prices
amzn_3pg['Replacement']=xchanges
amzn_3pg['Expected delivery']=xp_deliveries
amzn_3pg['Product availability']=prod_avail
amzn_3pg['Product description']=other_details
amzn_3pg['Product link']=prod_url

In [29]:
pd.set_option('display.max_colwidth',None)

In [30]:
pd.set_option('display.max_rows',None)

In [31]:
amzn_3pg

,Brand,Product,Rating,Number of ratings,Price,Replacement,Expected delivery,Product availability,Product description,Product link


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’

In [32]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('fruits',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/fruit_pics/fruits_'+str(i)+'.png')

### car pictures

In [33]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('cars',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/car_pics/car_'+str(i)+'.png')

### Machine Learning pics

In [34]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('Machine Learning',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/ml_pics/ml_'+str(i)+'.png')

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [35]:
def fetch_data():
    
    # get product title webelement
    title_wbe=driver.find_element_by_xpath("//span[@class='B_NuCI']")
    # append brand name
    brands.append(title_wbe.text.split(" ")[0])
    # append product name
    products.append(title_wbe.text.split(',')[0].replace("(",""))
    
    # get price webelement
    price_wbe=driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
    # append price
    prices.append(price_wbe.text)
    
    # get product url
    urls.append(driver.current_url)
            
    #get "specifications" text webelement which has all features in text
    features_wbe=driver.find_elements_by_xpath("//div[@class='_3k-BhJ']")
    # get text from all types of 'specifications' in one place
    for wbe in features_wbe:
        features.append(wbe.text.replace("\n","**").replace(",","^^"))
    
    # looping through all specification categories
    for j in range(0,len(features)):
        # check if specification category is 'General'
        if (features[j].split("**"))[0].lower()=='general':
            general=features[j].split("**")
            general_dict={}
            for i in general:
                if general.index(i)%2!=0:
                    general_dict[i.lower()]=general[general.index(i)+1]
            color.append(general_dict['color'])
        # check if specification category is 'Display Features'   
        elif (features[j].split("**"))[0].lower()=='display features':
            display=features[j].split("**")
            display_dict={}
            for i in display:
                if display.index(i)%2!=0:
                    display_dict[i.lower()]=display[display.index(i)+1]
            display_size.append(str(display_dict['display size']))
            display_reso.append(str(display_dict['resolution']))
        # check if specification category is 'OS & Processor Features'
        elif (features[j].split("**"))[0].lower()=='os & processor features':
            processor=features[j].split("**")
            processor_dict={}
            for i in processor:
                if processor.index(i)%2!=0:
                    processor_dict[i.lower()]=processor[processor.index(i)+1]
            if 'processor core' in processor_dict.keys(): 
                os_processors.append(str(processor_dict['operating system'])+str(processor_dict['processor core']))
            else:
                os_processors.append(str(processor_dict['operating system']))
        # check if specification category is 'Memory & Storage Features'
        elif (features[j].split("**"))[0].lower()=='memory & storage features':
            memory=features[j].split("**")
            memory_dict={}
            for i in memory:
                if memory.index(i)%2!=0:
                    memory_dict[i.lower()]=memory[memory.index(i)+1] 
            RAM.append(str(memory_dict['ram']))
            storage_ROM.append(str(memory_dict['internal storage']))
        # check if specification category is 'Camera Features'    
        elif (features[j].split("**"))[0].lower()=='camera features':
            cameras=features[j].split("**")
            camera_dict={}
            for i in cameras:
                if cameras.index(i)%2!=0:
                    camera_dict[i.lower()]=cameras[cameras.index(i)+1]
            primary_cameras.append(str(camera_dict['primary camera']))
            if 'secondary camera' in camera_dict.keys():
                secondary_cameras.append(str(camera_dict['secondary camera']))
            else:
                secondary_cameras.append(str("--"))
        # check if specification category is 'Battery & Power Features'    
        elif (features[j].split("**"))[0].lower()=='battery & power features':
            batt=features[j].split("**")
            batt_dict={}
            for i in batt:
                if batt.index(i)%2!=0:
                    batt_dict[i.lower()]=batt[batt.index(i)+1]
            battery.append(str(batt_dict['battery capacity']))

    features.clear()

In [36]:
open_url("https://www.flipkart.com")

# get web element for login email search bar
email_wbe=driver.find_element_by_xpath("//input[contains(@class,'_2IX_2-')]")#@class='_2IX_2- _2LYh3d VJZDxU'
# enter the email id
email_wbe.send_keys("8208507760")

#get web element for password bar
pwd_wbe=driver.find_element_by_xpath("//input[@type='password']")#class='_2IX_2- _3mctLh VJZDxU'
# enter the password
pwd_wbe.send_keys("Newpassword@72")

# get the webelement for login button
login_btn=driver.find_element_by_xpath("//button[@class='_2KpZ6l _2HKlqd _3AWRsL']")
#click on login button
login_btn.click()

#add timer sleep
time.sleep(3)

# get the web element for search bar
search_wbe=driver.find_element_by_xpath("//input[@name='q' and @placeholder='Search for products, brands and more']")
# write into search bar and pressing ENTER 
search_wbe.send_keys("Nokia 7.1 plus",Keys.ENTER)

#add timer sleep
time.sleep(3)


# get webelements for each result tile
result_wbe=driver.find_elements_by_xpath("//div[@class='_2kHMtA']//following::a")

urls=[]
urls.clear()
brands=[]
brands.clear()
products=[]
products.clear()
features=[]
features.clear()
prices=[]
prices.clear()
color=[]
color.clear()
display_size=[]
display_size.clear()
display_reso=[]
display_reso.clear()
os_processors=[]
os_processors.clear()
RAM=[]
RAM.clear()
storage_ROM=[]
storage_ROM.clear()
primary_cameras=[]
primary_cameras.clear()
secondary_cameras=[]
secondary_cameras.clear()
battery=[]
battery.clear()


for product_tile in result_wbe:
    #click on the product_tile one by one
    product_tile.click()
    
    #add time
    time.sleep(2)
    
    #check if new tab has opened
    if len(driver.window_handles)>1:    
        #switch driver to newopened tab
        driver.switch_to.window(driver.window_handles[1])
    else:
        break
    
    #click on the 'Read More' button
    read_more_btn=driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    read_more_btn.click()
    
    # add time
    time.sleep(2)
    
    fetch_data()
    
    # close the newly opened tab
    driver.close()
    
    # switch to main search page
    driver.switch_to.window(driver.window_handles[0])
    
    # add time
    time.sleep(2)

In [37]:
temp_df=pd.DataFrame({})
temp_df['Brand']=brands
temp_df['Product']=products
temp_df['Price']=prices
temp_df['Color']=color
temp_df['Display size']=display_size
temp_df['Resolution']=display_reso
temp_df['Operating system & processors']=os_processors
temp_df['RAM']=RAM
temp_df['Storage']=storage_ROM
temp_df['Primary Camera']=primary_cameras
temp_df['Secondary camera']=secondary_cameras
temp_df['Battery']=battery

In [38]:
temp_df

,Brand,Product,Price,Color,Display size,Resolution,Operating system & processors,RAM,Storage,Primary Camera,Secondary camera,Battery
0,Nokia,Nokia C20 Plus Blue,"₹9,999",Blue,16.51 cm (6.5 inch),1440 x 720 Pixels,Android 11Octa Core,3 GB,32 GB,8MP + 2MP,5MP Front Camera,5000 mAh
1,Nokia,Nokia C20 Plus Grey,"₹9,999",Grey,16.51 cm (6.5 inch),1440 x 720 Pixels,Android 11Octa Core,3 GB,32 GB,8MP + 2MP,5MP Front Camera,5000 mAh
2,Nokia,Nokia 6.1 Plus Blue,"₹15,000",Blue,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,6 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh
3,Nokia,Nokia 3.1 Plus Baltic,"₹10,440",Baltic,15.24 cm (6 inch),720 x 1440$$pixel,Android Oreo 8.1,3 GB,32 GB,13MP Rear Camera,--,3500 mAh
4,Nokia,Nokia C20 Plus Blue,"₹8,999",Blue,16.51 cm (6.5 inch),1440 x 720 Pixels,Android 11Octa Core,2 GB,32 GB,8MP + 2MP,5MP Front Camera,5000 mAh
5,Nokia,Nokia 5.1 Plus White,"₹13,199",White,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
6,Nokia,Nokia 7 Plus White & Copper,"₹15,999",White & Copper,15.24 cm (6 inch),2160 x 1080 Pixels,Android Oreo 8Octa Core,4 GB,64 GB,13MP Rear Camera,--,3800 mAh
7,Nokia,Nokia 3.1 Plus Charcoal,"₹7,088",Charcoal,15.24 cm (6 inch),720 x 1440 Pixels,Android Oreo Android Oreo 8.1Octa Core,3 GB,32 GB,13MP Rear Camera,--,3500 mAh
8,Nokia,Nokia 5.1 Plus Black,"₹13,199",Black,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
9,Nokia,Nokia 5.1 Plus Black,"₹9,990",Black,14.73 cm (5.8 inch),720 x 1520$$Pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,13MP + 5MP,8MP Front Camera,3060 mAh


In [39]:
temp_df.to_csv("Nokia 7.1 plus models.csv",float_format='%.2f',index=False)

In [40]:
pd.read_csv('Nokia 7.1 plus models.csv')

,Brand,Product,Price,Color,Display size,Resolution,Operating system & processors,RAM,Storage,Primary Camera,Secondary camera,Battery
0,Nokia,Nokia C20 Plus Blue,"₹9,999",Blue,16.51 cm (6.5 inch),1440 x 720 Pixels,Android 11Octa Core,3 GB,32 GB,8MP + 2MP,5MP Front Camera,5000 mAh
1,Nokia,Nokia C20 Plus Grey,"₹9,999",Grey,16.51 cm (6.5 inch),1440 x 720 Pixels,Android 11Octa Core,3 GB,32 GB,8MP + 2MP,5MP Front Camera,5000 mAh
2,Nokia,Nokia 6.1 Plus Blue,"₹15,000",Blue,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,6 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh
3,Nokia,Nokia 3.1 Plus Baltic,"₹10,440",Baltic,15.24 cm (6 inch),720 x 1440$$pixel,Android Oreo 8.1,3 GB,32 GB,13MP Rear Camera,--,3500 mAh
4,Nokia,Nokia C20 Plus Blue,"₹8,999",Blue,16.51 cm (6.5 inch),1440 x 720 Pixels,Android 11Octa Core,2 GB,32 GB,8MP + 2MP,5MP Front Camera,5000 mAh
5,Nokia,Nokia 5.1 Plus White,"₹13,199",White,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
6,Nokia,Nokia 7 Plus White & Copper,"₹15,999",White & Copper,15.24 cm (6 inch),2160 x 1080 Pixels,Android Oreo 8Octa Core,4 GB,64 GB,13MP Rear Camera,--,3800 mAh
7,Nokia,Nokia 3.1 Plus Charcoal,"₹7,088",Charcoal,15.24 cm (6 inch),720 x 1440 Pixels,Android Oreo Android Oreo 8.1Octa Core,3 GB,32 GB,13MP Rear Camera,--,3500 mAh
8,Nokia,Nokia 5.1 Plus Black,"₹13,199",Black,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
9,Nokia,Nokia 5.1 Plus Black,"₹9,990",Black,14.73 cm (5.8 inch),720 x 1520$$Pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,13MP + 5MP,8MP Front Camera,3060 mAh


### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [42]:
prompt=input("Enter a location: ")

open_url('https://www.google.com/maps/')

# get search bar webelement
search_bar=driver.find_element_by_xpath('//*[@id="searchboxinput"]')
# send city name
search_bar.send_keys(prompt,Keys.ENTER)
time.sleep(10)
current_url=driver.current_url
longitude=current_url.split('@')[1].split(',')[0]
lattitude=current_url.split('@')[1].split(',')[1]
print(prompt," Longitude: ",longitude,'"N')
print(prompt," Lattitude: ",lattitude,'"E')

Enter a location: noida
noida  Longitude:  28.5169834 "N
noida  Lattitude:  77.2580416 "E


6. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [45]:
open_url('https://www.digit.in')

# click on 'Best gaming laptops' link
driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a').click()

time.sleep(3)

# get laptop names
names=[]
names.clear()

names_wbe=driver.find_elements_by_xpath("//h3")

for i in names_wbe:
    names.append(i.text)
    
time.sleep(2)

# get specifications box weblements
spec_wbe=driver.find_elements_by_xpath("//div[@class='Spcs-details']")

specs=[]
specs.clear()

for wbe in spec_wbe:
    specs.append(wbe.text)

In [46]:
names

['ALIENWARE AREA 51M R2',
 'ALIENWARE M15 R3',
 'ASUS ROG STRIX SCAR 15',
 'ASUS ROG ZEPHYRUS G14',
 'LENOVO LEGION 5I',
 'ASUS ROG ZEPHYRUS DUO 15',
 'ACER ASPIRE 7 GAMING']

In [47]:
specs

['SPECIFICATION\nOS : Windows 10 Home\nDisplay : 17.3" (1920 x 1080)\nProcessor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 4.1\nDimension : 27.65 x 402.6 x 319.14\nGraphics Processor : Intel® UHD Graphics 630',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 15.6" (3840 x 2160)\nProcessor : 10th Generation Intel® Core™ i9-10980HK | NA\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : NA\nDimension : NA\nGraphics Processor : NA\nPrice : ₹341990',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 15.6" (1920 x 1080)\nProcessor : AMD Ryzen™ 9 5900HX | 3.3 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 2.30\nDimension : 35.4 x 25.9 x 2.26\nGraphics Processor : NVIDIA® GeForce RTX™ 3070',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 14" (1920 x 1080)\nProcessor : AMD 3rd Generation Ryzen 9 | 3.3 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 1.65\nDimension : 32.5 x 22.1 x 1.8\nGraphics Processor : NVIDIA GeForce RTX 2060',
 'SPECIFICATION\nOS : W

In [48]:
specs[0]

'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 17.3" (1920 x 1080)\nProcessor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 4.1\nDimension : 27.65 x 402.6 x 319.14\nGraphics Processor : Intel® UHD Graphics 630'

In [49]:
temp_specs=[]
for specs in specs:
    temp_specs.append(specs.split('\n'))
temp_specs[0]

['SPECIFICATION',
 'OS : Windows 10 Home',
 'Display : 17.3" (1920 x 1080)',
 'Processor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz',
 'Memory : 1 TB SSD/16 GBGB DDR4',
 'Weight : 4.1',
 'Dimension : 27.65 x 402.6 x 319.14',
 'Graphics Processor : Intel® UHD Graphics 630']

In [50]:
temp_specs

[['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 17.3" (1920 x 1080)',
  'Processor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : 4.1',
  'Dimension : 27.65 x 402.6 x 319.14',
  'Graphics Processor : Intel® UHD Graphics 630'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 15.6" (3840 x 2160)',
  'Processor : 10th Generation Intel® Core™ i9-10980HK | NA',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : NA',
  'Dimension : NA',
  'Graphics Processor : NA',
  'Price : ₹341990'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 15.6" (1920 x 1080)',
  'Processor : AMD Ryzen™ 9 5900HX | 3.3 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : 2.30',
  'Dimension : 35.4 x 25.9 x 2.26',
  'Graphics Processor : NVIDIA® GeForce RTX™ 3070'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 14" (1920 x 1080)',
  'Processor : AMD 3rd Generation Ryzen 9 | 3.3 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'W

In [51]:
temp_specs[0][3][0:3]

'Pro'

In [52]:
os=[]
dis=[]
processor=[]
memory=[]
weight=[]
dimension=[]
graphic=[]
price=[]

In [53]:
for j in temp_specs:
    if len(j)==8:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                price.append('--')
                continue
    elif len(j)==9:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pri':
                price.append(i.split(':')[-1])
                time.sleep(2)
                continue

In [54]:
df_digit=pd.DataFrame({})
df_digit['Model Name']=names
df_digit['Operating System']=os
df_digit['Display']=dis
df_digit['Processor']=processor
df_digit['Memory']=memory
df_digit['Weight']=weight
df_digit['Dimension']=dimension
df_digit['Graphics Processor']=graphic
df_digit['Price']=price

In [55]:
df_digit

,Model Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,--
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,₹341990
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,--
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,--
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,--
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,--
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,--


In [56]:
df_digit.to_csv('Best gaming laptops.csv',float_format='%.2f',index=False)

In [57]:
df_temp=pd.read_csv('Best gaming laptops.csv')

In [58]:
df_temp

,Model Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,--
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,₹341990
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,--
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,--
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,--
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,--
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,--
